<a href="https://colab.research.google.com/github/arashsajjadi/Bigdata/blob/main/Shahid%20Beheshti%20University%20course/Assignments/First%20series/Question5/SparkTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h1 dir=rtl>
«تمرین برنامه‌نویسی اول»
</h1>
<h3 dir=rtl>
 نصب اسپارک و پردازش‌های اولیه با آن
</h3>
</center>

<div dir=rtl>
ابتدا بسته ‌اسپارک را در پایتون نصب کنید:
</div>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<div dir=rtl>
سپس کتابخانه‌های مربوطه را import کنید.
</div>

In [2]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

<div dir=rtl>
با دستورات ذیل یک session و context برای اسپارک ایجاد می‌کنیم. 
</div>

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

<div dir=rtl>
دستور spark را اجرا کنید تا مطمئن شوید که همه‌چیز به درستی نصب و تنظیم شده است
</div>

In [4]:
spark

<div dir=rtl>
در این تمرین یک دیتاست معروف به نام Record Linkage را دانلود می‌کنیم. 

سوال اول: از لینک ذیل درباره این دیتاست مطالعه کنید و در یک پاراگراف آنرا معرفی کنید.
</div>
<div>
<a href="https://archive.ics.uci.edu/ml/datasets/record+linkage+comparison+patterns">https://archive.ics.uci.edu/ml/datasets/record+linkage+comparison+patterns</a>
</div>

---------------------------------------------

اطلاعات مربوط به مشخصات فردی اشخصاص دو یک دوره سه ساله بین سال‌های ۲۰۰۵ تا ۲۰۰۸ است. این داده‌ها از طریق درج‌های تکراری از حدود ۱۰۰ هزار نمونه جمع‌آوری شده است. جفت داده‌ها توسط چندین مستند ساز با عنوان‌های «مطابقت» یا «غیر منطبق» طبقه‌بندی شدند. سپس برای پردازش راحت‌تر فقط و فقط جفت‌هایی انتخاب شدند که شرایط تطابق را داشتند. 
در نهایت نتایج شش شرط تشابه با یک دیگر ادغام شدند و داده‌های نهایی را به وجود آوردند.

In [5]:
!curl -L -o donation.zip https://bit.ly/1Aoywaq
!unzip donation.zip
!unzip 'block_*.zip'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   163  100   163    0     0   2810      0 --:--:-- --:--:-- --:--:--  2810
100 53.8M  100 53.8M    0     0  25.9M      0  0:00:02  0:00:02 --:--:-- 33.3M
Archive:  donation.zip
 extracting: block_10.zip            
 extracting: block_1.zip             
 extracting: block_2.zip             
 extracting: block_3.zip             
 extracting: block_4.zip             
 extracting: block_5.zip             
 extracting: block_6.zip             
 extracting: block_7.zip             
 extracting: block_8.zip             
 extracting: block_9.zip             
  inflating: documentation           
  inflating: frequencies.csv         
Archive:  block_1.zip
  inflating: block_1.csv             

Archive:  block_4.zip
  inflating: block_4.csv             

Archive:  block_10.zip
  inflating: block_10.csv            

Archive:  block_5.zip

<div dir=rtl>
در ادامه، همه فایل‌های csv مربوط به این دیتاست را در یک دیتافریم لود می‌کنیم:
</div>

In [6]:
df0 = spark.read.csv("block*.csv")

<div dir=rtl>
با دستور ذیل تعداد رکوردهای این دیتافریم را بدست می‌آوریم:
</div>

In [7]:
df0.count()

5749142

<div dir=rtl>
با این دستور شمای این دیتافریم را مشاهده می‌کنیم. دقت کنید که نام ستونها و نوع آنها همگی به صورت خودکار تولید شده‌اند
</div>

In [8]:
df0.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)



<div dir=rtl>
برای نمایش ۵ سطر اول این دیتافریم از دستور ذیل استفاده می‌کنیم
</div>

In [9]:
df0.show(5)

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
|  _c0|  _c1|         _c2|         _c3|         _c4|         _c5|    _c6|   _c7|   _c8|   _c9|   _c10|    _c11|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
| 3148| 8326|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|14055|94934|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|33948|34740|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|  946|71870|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--

<div dir=rtl>
حال، مجدد فایل‌های csv را با پارامترهای دیگری لود می‌کنیم:
</div>

In [10]:
parsed = spark.read.csv("block*.csv", header=True, inferSchema=True, nullValue="?")

In [11]:
parsed.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [12]:
parsed.show(5)

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|14055|94934|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|33948|34740|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|  946|71870|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|64880|71676|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--

<div dir=rtl>
سوال دوم:
با تغییر انجام شده در نحوه خواندن فایل‌های csv چه تغییری در DataFrame جدید نسبت به حالت قبل ایجاد شده است؟
</div>




1.   عنوان ستون‌ها به عنوان یک رکورد در داده‌ها نیستند و تبدیل یه عنوان شدند. 
2.   داده‌های تهی مشخصاً با همین عنوان مشخص شدند.
3. بدیهی است که مورد اول باعث می شود که در گزارش گرفتن از داده‌ها یک ثبت کم‌تر شود.
4. ستون‌ها از حالت استرینگ خارج شدند



<div dir=rtl>
در ادامه، دستور ذیل را اجرا کنید:
</div>


In [13]:
parsed.groupBy('is_match').count().show()

+--------+-------+
|is_match|  count|
+--------+-------+
|    true|  20931|
|   false|5728201|
+--------+-------+



<div dir=rtl>
سوال سوم: 
دستور بالا چه کاری انجام می‌دهد؟
</div>

ما 5.749.132 داده داشتیم که با دستور فوق آنهایی که با یکدیگر تطابق داشتند را شناسایی کردیم و می دانیم 20.931 از آنها با یکدیگر در تطابق هستند.

<div dir=rtl>
برای نمایش اطلاعات آماری از دیتاست از دستور ذیل استفاده می‌کنیم:
</div>

In [14]:
summary = parsed.describe()
summary.show()

+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|summary|              id_1|              id_2|      cmp_fname_c1|      cmp_fname_c2|       cmp_lname_c1|       cmp_lname_c2|            cmp_sex|             cmp_bd|             cmp_bm|            cmp_by|            cmp_plz|
+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|  count|           5749132|           5749132|           5748125|            103698|            5749132|               2464|            5749132|            5748337|            5748337|           5748337|            5736289|
|   mean| 33324.48559643438| 66587.43558331935|0.7129024704425707| 0.900017671890335|0.3156278193076

<div dir=rtl>
سوال چهارم:
چرا مقادیر count در ستون‌های مختلف متفاوت است؟
</div>

درایه‌های تهی پاسخ اصلی است.
اما در ضرب ماتریس‌ها هم داشتیم که داده های صفر را ذخیره نمی‌کردند که البته در اینجا این نکته صدق نخواهد کرد. در انتهای این نوتبوک بیشتر توضیح خواهم داد.
در واقع در اینجا صرفا اطلاعاتی از آن ستون خاص در داده مذکور وجود ندارد و این باث میشد درایه تهی باشد

<div dir=rtl>
می‌توان آماره‌های مد نظر را روی دو دسته رکوردهایی که مقادیر فیلد is_matched آنها متفاوت است اجرا کرد. 

در اینجا به نحوه  فیلتر  بر اساس این فیلد توجه کنید.
</div>

In [15]:
matches = parsed.where("is_match = true")
match_summary = matches.describe()

misses = parsed.filter(col("is_match") == False) 
miss_summary = misses.describe()

In [16]:
matches.describe()

DataFrame[summary: string, id_1: string, id_2: string, cmp_fname_c1: string, cmp_fname_c2: string, cmp_lname_c1: string, cmp_lname_c2: string, cmp_sex: string, cmp_bd: string, cmp_bm: string, cmp_by: string, cmp_plz: string]

<div dir=rtl>
روی دیتافریم ایجاد شده می‌توان انواع پیش‌پردازش‌ها را انجام داد. 
به عنوان نمونه فرض کنید می‌خواهیم تمام مقادیر null را با مقدار صفر جایگزین کنیم.

سوال پنجم: برای انجام این کار از چه دستوری باید استفاده کرد؟

</div>

In [17]:
# Fill the null values  with 0
parsed.na.fill(0)
####################################################
#In the following report, you can also see that the null values were replaced with 0

summary = parsed.na.fill(0).describe()
summary.show()

+-------+------------------+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+------------------+-------------------+-------------------+
|summary|              id_1|              id_2|       cmp_fname_c1|       cmp_fname_c2|       cmp_lname_c1|        cmp_lname_c2|            cmp_sex|             cmp_bd|            cmp_bm|             cmp_by|            cmp_plz|
+-------+------------------+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+------------------+-------------------+-------------------+
|  count|           5749132|           5749132|            5749132|            5749132|            5749132|             5749132|            5749132|            5749132|           5749132|            5749132|            5749132|
|   mean| 33324.48559643438| 66587.43558331935| 0.7127776006730584|0.01623376059893632|0

 در گزارش فوق نیز میتوانید ببینید که مقادیر تهی با ۰ جایگزین شدند که دو نکته حائز اهمیت را میتوان مشاهده کرد.
 
 ۱. تعداد شمارش هر ستون برابر کل داده‌ها شد

 ۲. شاخص‌های آماری نسبت به ورژن اولیه متفاوت شد